# Lecture 19: Join Operations
This lecture will focus on taking a set of csv files and making a relational database from them. Then we will use that relational database to generate some useful composite tables.

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import pdb

## Create a connection to an "in memory" database
Make a connection to the syllabus database file.

In [2]:
def create_connection(db_file):
    """ create a database connection to the syllabus db """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as error:
        print(error)

In [3]:
syllabus_file = ':memory:'
conn = create_connection(syllabus_file)

In [6]:
# create the db from the schema
with open("syllabus-schema.sql", "r") as schema_f:
    schema = schema_f.read()
    conn.executescript(schema)

## Populate the database with the csv tables

In [7]:
def populate_table(csv_file, conn, table_name):
    """ populates a db table with data from csv_file """

    csv_df = pd.read_csv(csv_file)
    csv_df.to_sql(table_name, conn, if_exists="append", index=False)

In [8]:
# populate the db using pandas
populate_table("outcomes.csv", conn, "outcomes")
populate_table("assessments.csv", conn, "assessments")
populate_table("lectures.csv", conn, "lectures")
populate_table("schedule.csv", conn, "schedule")

## Join the lectures and schedule tables to make the lecture schedule

In [9]:
def lecture_schedule(conn):
    """ creates a csv file containing the schedule """

    select_stmt = """
    SELECT schedule.id,
           schedule.date,
           schedule.day_of_week,
           lectures.unit,
           lectures.name,
           lectures.reading
    FROM schedule
    LEFT JOIN lectures AS lectures ON schedule.lecture_id = lectures.id;
    """

    schedule_df = pd.read_sql(select_stmt, conn, parse_dates={'date':'%m/%d/%Y'})
    schedule_df.columns = ['No.', 'Date', 'DoW', 'Unit', 'Topic', 'Reading']
    schedule_df["Date"] = schedule_df["Date"].dt.strftime("%m/%d/%Y")
    schedule_df.fillna(value='', inplace=True)

    return schedule_df

In [10]:
# print the formatted schedule
lecture_schedule_df = lecture_schedule(conn)
with open("lecture_schedule.tex", "w") as tex_f:
    lecture_schedule_df.to_latex(tex_f, column_format='@{}rlllll@{}',
                                index=False)

## Join the schedule and assessment tables to create the assessment schedule

In [22]:
def assessment_schedule(conn):
    """ creates a dataframe containing the assessment schedule """

    dist_stmt = """
    SELECT schedule.id,
           schedule.date,
           assessments_distributed.name as dist
    FROM schedule
    INNER JOIN assessments AS assessments_distributed 
    ON schedule.assessment_distributed_id = assessments_distributed.id
    """

    coll_stmt = """
    SELECT schedule.id,
           schedule.date,
           assessments_collected.name as coll
    FROM schedule
    INNER JOIN assessments AS assessments_collected 
    ON schedule.assessment_collected_id = assessments_collected.id
    """

    # Run SELECT to extract data frames
    dist_df = pd.read_sql(dist_stmt, conn, parse_dates={'date': '%m/%d/%Y'})
    coll_df = pd.read_sql(coll_stmt, conn, parse_dates={'date': '%m/%d/%Y'})

    # Combine using OUTER join
    schedule_df = pd.merge(dist_df, coll_df, on=['id', 'date'], how='outer', sort=True)
    schedule_df.columns = ['No', 'Date', 'Distr.', 'Coll.']
    schedule_df["Date"] = schedule_df["Date"].dt.strftime("%m/%d/%Y")
    schedule_df.fillna(value='', inplace=True)

    return schedule_df

In [23]:
assessment_schedule_df = assessment_schedule(conn)
with open("assessment_schedule.tex", "w") as tex_f:
    assessment_schedule_df.to_latex(tex_f, column_format='@{}rlll@{}',
                                    index=False)

## Close the connection to the database

In [13]:
conn.close()

# Getting Started on Pset 5

In [3]:
conn=sqlite3.connect("baseball.db")
c = conn.cursor()

### Part 1
Print all of the table names in the database. You should only show the names of the tables and there should be one table name per line.

In [34]:
#find all tables names in the database
c.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in c.fetchall():
    print(name[0])

Master
sqlite_sequence
Teams
TeamsFranchises
TeamsHalf
Batting
Pitching
Fielding
FieldingOF
Salaries
Managers
ManagersHalf
AllstarFull
AwardsPlayers
AwardsSharePlayers
AwardsManagers
AwardsShareManagers
HallOfFame
BattingPost
FieldingPost
PitchingPost
SeriesPost
Schools
SchoolsPlayers
xref_stats
Appearances


### Part 2
Print the column names for the Salaries table.

In [35]:
c.execute("SELECT * From Salaries")
# Using a list comprehension here
names = [description[0] for description in c.description]
print(names)

['yearID', 'teamID', 'lgID', 'playerID', 'salary']


### Part 3
Print the first ten records in the Salaries table.

In [36]:
#find column names of Salaries table
c.execute("SELECT * From Salaries")
print(c.fetchmany(10))# first 10 rows
# Notice that a list of tuples is returned

[(1980, 'TOR', 'AL', 'stiebda01', 55000.0), (1981, 'NYA', 'AL', 'jacksre01', 588000.0), (1981, 'TOR', 'AL', 'stiebda01', 85000.0), (1982, 'TOR', 'AL', 'stiebda01', 250000.0), (1983, 'TOR', 'AL', 'stiebda01', 450000.0), (1984, 'TOR', 'AL', 'stiebda01', 650000.0), (1985, 'ATL', 'NL', 'barkele01', 870000.0), (1985, 'ATL', 'NL', 'bedrost01', 550000.0), (1985, 'ATL', 'NL', 'benedbr01', 545000.0), (1985, 'ATL', 'NL', 'campri01', 633333.0)]


## Problem B: Grouping and Sorting
### Part 1
Print the first 10 teamIDs and average salaries from the Salary table for 2010, where 2010 is a variable.

In [4]:
y = ('2010',)
c.execute('SELECT teamID, AVG(salary) From Salaries WHERE yearID = ? GROUP BY teamID', y)
for r in c.fetchall():
    print((r[0], r[1]))

('ARI', 2335314.076923077)
('ATL', 3126802.4444444445)
('BAL', 3138942.3076923075)
('BOS', 5601632.172413793)
('CHA', 4058846.153846154)
('CHN', 5429962.962962963)
('CIN', 2760059.3076923075)
('CLE', 2110481.5862068967)
('COL', 2904379.3103448274)
('DET', 4550552.888888889)
('FLO', 2112211.814814815)
('HOU', 3298410.714285714)
('KCA', 2644637.407407407)
('LAA', 3619443.6551724137)
('LAN', 3531778.3703703703)
('MIL', 2796837.172413793)
('MIN', 3484255.9285714286)
('NYA', 8253335.56)
('NYN', 4800819.357142857)
('OAK', 1726715.625)
('PHI', 5068870.678571428)
('PIT', 1294185.1851851852)
('SDN', 1453819.2307692308)
('SEA', 3089642.8571428573)
('SFN', 3522904.75)
('SLN', 3741630.04)
('TBA', 2663832.2592592593)
('TEX', 1905191.1724137932)
('TOR', 2074466.6666666667)
('WAS', 2046666.6666666667)


### Part 2
Print the playerID and salary for the player with this highest salary in 2010. Make the year a variable in your execute statement. You should have one execute statement and one print statement.

In [38]:
c.execute('SELECT MAX(salary), playerID From Salaries WHERE yearID = ?', y)
print(c.fetchall())

[(33000000.0, 'rodrial01')]


### Part 3
Print the teamID and total salary for each team in 2010. Sort by descending total salary. Make the year a variable in your execute statement.

In [39]:
c.execute('SELECT teamID, SUM(salary) FROM Salaries WHERE yearID = ? GROUP BY teamID ORDER BY SUM(salary) DESC',y)
for r in c.fetchall():
    print((r[0], r[1]))

('NYA', 206333389.0)
('BOS', 162447333.0)
('CHN', 146609000.0)
('PHI', 141928379.0)
('NYN', 134422942.0)
('DET', 122864928.0)
('CHA', 105530000.0)
('LAA', 104963866.0)
('SFN', 98641333.0)
('MIN', 97559166.0)
('LAN', 95358016.0)
('SLN', 93540751.0)
('HOU', 92355500.0)
('SEA', 86510000.0)
('ATL', 84423666.0)
('COL', 84227000.0)
('BAL', 81612500.0)
('MIL', 81108278.0)
('TBA', 71923471.0)
('CIN', 71761542.0)
('KCA', 71405210.0)
('TOR', 62234000.0)
('WAS', 61400000.0)
('CLE', 61203966.0)
('ARI', 60718166.0)
('FLO', 57029719.0)
('OAK', 55254900.0)
('TEX', 55250544.0)
('SDN', 37799300.0)
('PIT', 34943000.0)
